In [ ]:
import pandas as pd
import math
import numpy as np
import copy
from collections import Counter
from scipy.special import comb
import itertools
from cvxopt import solvers, blas, matrix, spmatrix, spdiag, log, div
transdf = pd.read_csv('1000000_normal_64_1.csv')
def mean_absolute_error(freq, est_freq):
    return (abs(freq - est_freq)).mean()
def mean_relative_error(freq, est_freq):
    freq /= sum(freq)
    est_freq /= sum(est_freq)
    return (abs(freq - est_freq)/(freq + np.finfo(np.float32).eps)).mean()
def jaccard_index(freq, est_freq, k):
    top_k = np.argpartition(-freq, k)[:k]
    est_top_k = np.argpartition(-est_freq, k)[:k]
    return len(np.intersect1d(top_k, est_top_k))/len(np.union1d(top_k, est_top_k))
def mean_square_error(freq, est_freq):
    return (np.square(freq - est_freq)).mean()
def root_mean_square_error(freq, est_freq):
    return np.sqrt((np.square(freq - est_freq)).mean())

total_n = len(transdf)
l = 0.2 #Privacy budget difference
alpha_1 = 0.7
alpha_2 = 0.03
lamda = 3
total_dim = 10
epsilon_dim = 10
fan_out = 4
c_domain = 64
lamda_dim = 2

# epsilon_type = 'uniform1'
# epsilon_type = 'normal1'
# epsilon_type = 'laplace1'
epsilon_type = 'prob1'

# index = [[-1,0,1,2,3,4],[0,-1,5,6,7,8],[1,5,-1,9,10,11],[2,6,9,-1,12,13],[3,7,10,12,-1,14],[4,8,11,13,14,-1]]
index = [[-1,0,1,2,3,4,5,6,7,8],[0,-1,9,10,11,12,13,14,15,16],[1,9,-1,17,18,19,20,21,22,23],[2,10,17,-1,24,25,26,27,28,29],
         [3,11,18,24,-1,30,31,32,33,34],[4,12,19,25,30,-1,35,36,37,38],[5,13,20,26,31,35,-1,39,40,41],[6,14,21,27,32,36,39,-1,42,43],
         [7,15,22,28,33,37,40,42,-1,44],[8,16,23,29,34,38,41,43,44,-1]]

member1 = np.zeros(epsilon_dim,dtype=int)
member = np.zeros(epsilon_dim,dtype=int)
value_counts = Counter(transdf[epsilon_type])
for value, count in value_counts.items():
    member1[value-1] = count//(total_dim+comb(total_dim,2))
for i in range(epsilon_dim):
    member[i] = np.sum(member1[i:epsilon_dim])
    
total_group_LHIO = comb(total_dim,2)*(math.log(c_domain)/math.log(fan_out)+1)*(math.log(c_domain)/math.log(fan_out)+1)
member_LHIO = np.zeros(epsilon_dim,dtype=int)
value_counts = Counter(transdf[epsilon_type])
for value, count in value_counts.items():
    member_LHIO[value-1] = count//total_group_LHIO

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(64)
plt.hist(transdf['attribute9'], bins = len(x))

In [ ]:
print(np.max(transdf))

In [ ]:
def NormSub(num,est_dist):
    estimates = np.copy(est_dist).astype('float64')
    while (np.abs(sum(estimates) - num) > 1) or (estimates < 0).any():
        if (estimates <= 0).all():
            estimates[:] = num / estimates.size
            break
        estimates[estimates < 0] = 0
        total = sum(estimates)
        mask = estimates > 0
        diff = (num - total) / sum(mask)
        estimates[mask] = estimates[mask]+diff
    return estimates


In [ ]:
def choose_g(str1,str2,k):
    ga = np.zeros(epsilon_dim,dtype=int)
    for j in range(epsilon_dim):
        if str1 == 'TDG' and str2 == 'g2':
            n = total_n//epsilon_dim
            m = comb(k,2)
            x1 = 2 * alpha_2 * (math.exp((j+1)*l) - 1)
            x2 = n / (m * math.exp((j+1)*l))
            x3 = math.sqrt(x2)
            value = math.sqrt(x1 * x3)
            ga[j] = 2 ** int(math.log2(value))
        elif str1 == 'HDG' and str2 == 'g2':
            n = total_n//epsilon_dim
            m = k+comb(k,2)
            x1 = 2 * alpha_2 * (math.exp((j+1)*l) - 1)
            x2 = n / (m * math.exp((j+1)*l))
            x3 = math.sqrt(x2)
            value = math.sqrt(x1 * x3)
            ga[j] = 2 ** int(math.log2(value))
        else: #HDG,g1
            n = total_n//epsilon_dim
            m = k + comb(k,2)
            x1 = n * (math.exp((j+1)*l) - 1) * (math.exp((j+1)*l) - 1) *alpha_1 * alpha_1
            x2 = 2 * m * math.exp((j+1)*l)
            value = math.pow(x1 / x2, 1 / 3)
            ga[j] = 2 ** int(math.log2(value))

        if ga[j] < 2:
            ga[j] = 2
        if ga[j] > c_domain:
            ga[j] = c_domain
    ga = np.round(ga)
    return ga

In [ ]:
def OUE(group_data, attr, k, optimal):
    ga = choose_g('HDG','g1',total_dim)
    true_freq = []
    private_freq = []
    private_group = []
    # Initializing a zero-vector
    block = np.zeros(epsilon_dim,dtype=int)
    # true_freq = np.zeros((10,k),dtype=int)
    # private_freq = np.zeros((10,k),dtype=int)    
    block = np.round(k/ga) #the length of each cell]
    tmpk = k
    for j in range(epsilon_dim):
        data = group_data.loc[group_data[epsilon_type]==j+1][[attr]].values
        input_data = data // block[j]
        k = math.ceil(tmpk/block[j])
        private_data = np.zeros((len(input_data),k),dtype=int)
        true_freq_one = np.zeros(k,dtype=int)
        private_freq_one = np.zeros(k,dtype=int)   
        # Unary encoding
        input_ue_data = np.zeros((len(input_data),k),dtype=int)
        # UE perturbation function
        for i in range(len(input_data)):
            # Unary encoding
            input_ue_data[i][int(input_data[i])] = 1
            true_freq_one += input_ue_data[i]
            # Symmetric parameters (p+q = 1)
            p = np.exp((j+1)*l/2) / (np.exp((j+1)*l/2) + 1)
            q = 1 - p
            # Optimized parameters
            if optimal:
                p = 1 / 2
                q = 1 / (np.exp((j+1)*l) + 1)
            private_data[i] = np.where(input_ue_data[i] == 1, 
                np.random.binomial(1, p, k), 
                np.random.binomial(1, q, k))
            private_freq_one += private_data[i]
        true_freq.append(true_freq_one)
        private_freq.append(private_freq_one)
        private_group.append(private_data)
        
        private_freq[j] = np.array((private_freq[j] - len(data)*q) / (p-q))
        private_freq[j] = np.round(NormSub(len(data),private_freq[j]))
        # # Re-normalized estimated frequency
        # norm_est_freq = np.nan_to_num(est_freq / sum(est_freq))
    # true_final_freq = np.sum(true_freq,axis = 0)
    # private_final_freq = np.sum(private_freq,axis = 0)
    # MAE = mean_absolute_error(true_final_freq,private_final_freq)
    # # MRE = mean_relative_error(true_final_freq,private_final_freq)
    # MSE = mean_square_error(true_final_freq,private_final_freq)
    # RMSE = root_mean_square_error(true_final_freq,private_final_freq)
    # print(MAE,MSE,RMSE)
    # print(true_freq,'\n',private_freq,'\n',true_final_freq,'\n',private_final_freq)
    return true_freq,private_freq,private_group
def two_OUE(group_data, attr1, attr2, k1, k2, optimal):
    ga1 = choose_g('HDG','g2',total_dim)
    ga2 = choose_g('HDG','g2',total_dim)
    block1 = np.zeros(epsilon_dim,dtype=int)
    block2 = np.zeros(epsilon_dim,dtype=int)

    true_freq = []
    private_freq = []
    private_group = []
    block1 = np.round(k1/ga1)
    block2 = np.round(k2/ga2)

    tmpk1 = k1
    tmpk2 = k2
    for j in range(epsilon_dim):
        data1 = group_data.loc[group_data[epsilon_type]==j+1][[attr1]].values
        input_data1 = data1 // block1[j]
        k1 = math.ceil(tmpk1/block1[j])
        data2 = group_data.loc[group_data[epsilon_type]==j+1][[attr2]].values
        input_data2 = data2 // block2[j]
        k2 = math.ceil(tmpk2/block2[j])
        private_freq_one = np.zeros(k1*k2,dtype=int)
        true_freq_one = np.zeros(k1*k2,dtype=int)
        private_data = np.zeros((len(input_data1),k1*k2),dtype=int)
        # Unary encoding
        input_ue_data = np.zeros((len(input_data1),k1*k2),dtype=int)
        # UE perturbation function
        for i in range(len(data1)):
            # Unary encoding
            true_data = input_data1[i]*k2 + input_data2[i]
            input_ue_data[i][int(true_data)] = 1
            true_freq_one += input_ue_data[i]
            # Symmetric parameters (p+q = 1)
            p = np.exp((j+1)*l/2) / (np.exp((j+1)*l/2) + 1)
            q = 1 - p
            # Optimized parameters
            if optimal:
                p = 1 / 2
                q = 1 / (np.exp((j+1)*l) + 1)
            private_data[i] = np.where(input_ue_data[i] == 1, 
                np.random.binomial(1, p, k1*k2), 
                np.random.binomial(1, q, k1*k2))
            private_freq_one += private_data[i]
        true_freq.append(true_freq_one)
        private_freq.append(private_freq_one)
        private_group.append(private_data)

        #Aggregation
        # Ensure non-negativity of estimated frequency
        private_freq[j] = np.array((private_freq[j] - len(data1)*q) / (p-q))
        # private_freq[j] = np.nan_to_num(private_freq[j] / sum(private_freq[j]))
        private_freq[j] = np.round(NormSub(len(data1),private_freq[j]))
        # # Re-normalized estimated frequency
        # norm_est_freq = np.nan_to_num(est_freq / sum(est_freq))
    # true_final_freq = np.sum(true_freq,axis = 0)
    # private_final_freq = np.sum(private_freq,axis = 0)

    # MAE = mean_absolute_error(true_final_freq,private_final_freq)
    # # MRE = mean_relative_error(true_final_freq,private_final_freq)
    # MSE = mean_square_error(true_final_freq,private_final_freq)
    # RMSE = root_mean_square_error(true_final_freq,private_final_freq)
    # print(MAE,MSE,RMSE)
    # print(true_freq,'\n',private_freq,'\n',true_final_freq,'\n',private_final_freq)
    return true_freq,private_freq,private_group

In [ ]:
def two_LHIO(group_data, attr1, attr2, k1, k2, optimal):
    true_freq = []
    private_freq = []
    block1 = c_domain/k1
    block2 = c_domain/k2

    for j in range(epsilon_dim):
        data1 = group_data.loc[group_data[epsilon_type]==j+1][[attr1]].values
        input_data1 = data1 // block1
        data2 = group_data.loc[group_data[epsilon_type]==j+1][[attr2]].values
        input_data2 = data2 // block2
        private_freq_one = np.zeros(k1*k2,dtype=int)
        true_freq_one = np.zeros(k1*k2,dtype=int)
        private_data = np.zeros((len(input_data1),k1*k2),dtype=int)
        # Unary encoding
        input_ue_data = np.zeros((len(input_data1),k1*k2),dtype=int)
        # UE perturbation function
        for i in range(len(data1)):
            # Unary encoding
            true_data = input_data1[i]*k2 + input_data2[i]
            input_ue_data[i][int(true_data)] = 1
            true_freq_one += input_ue_data[i]
            # Symmetric parameters (p+q = 1)
            p = np.exp((j+1)*l/2) / (np.exp((j+1)*l/2) + 1)
            q = 1 - p
            # Optimized parameters
            if optimal:
                p = 1 / 2
                q = 1 / (np.exp((j+1)*l) + 1)
            private_data[i] = np.where(input_ue_data[i] == 1, 
                np.random.binomial(1, p, k1*k2), 
                np.random.binomial(1, q, k1*k2))
            private_freq_one += private_data[i]
        true_freq.append(true_freq_one)
        private_freq.append(private_freq_one)

        #Aggregation
        # Ensure non-negativity of estimated frequency
        private_freq[j] = np.array((private_freq[j] - len(data1)*q) / (p-q))
        # private_freq[j] = np.nan_to_num(private_freq[j] / sum(private_freq[j]))
        private_freq[j] = np.round(NormSub(len(data1),private_freq[j]))
    return true_freq,private_freq

In [ ]:
def DR(group,optimal):
    group_add = copy.deepcopy(group)
    for level in range(epsilon_dim): 
        for ee in range(level+1,epsilon_dim):
            num = len(group[ee])
            compressed_data = []
            qs = math.exp((ee+1)*l/2) / (math.exp((ee+1)*l/2) + 1)*2 - 1
            qz = math.exp((level+1)*l/2) / (math.exp((level+1)*l/2) + 1)*2 - 1
            # Optimized parameters
            if optimal:
                qs = 1/2 - 1 / (math.exp((ee+1)*l) + 1)
                qz = 1/2 - 1 / (math.exp((level+1)*l) + 1)
            pp = (qs+qz)/(2*qs)
            qq = (qs-qz)/(2*qs)

            if len(group[ee][0]) == len(group[level][0]):
                compressed_data = group[ee]
            else:
                for m in range(0, len(group[ee][0]), len(group[ee][0])//len(group[level][0])):
                    # 将每四列合并为一个元素
                    compressed_column = np.any(group[ee][:, m:m+len(group[ee][0])//len(group[level][0])], axis=1).astype(int)
                    compressed_column = np.expand_dims(compressed_column, axis=1)
                    if len(compressed_data) == 0:
                        compressed_data = compressed_column
                    else:
                        compressed_data = np.concatenate((compressed_data, compressed_column), axis=1)
            for i in range(num):
                compressed_data[i] = np.where(compressed_data[i] == 1, 
                    np.random.binomial(1, pp, len(compressed_data[i])), 
                    np.random.binomial(1, qq, len(compressed_data[i])))

            group_add[level] = np.row_stack((group_add[level], compressed_data))
        p = np.exp((level+1)*l/2) / (np.exp((level+1)*l/2) + 1)
        q = 1 - p
        # Optimized parameters
        if optimal:
            p = 1 / 2
            q = 1 / (np.exp((level+1)*l) + 1)
        member_group = group_add[level].shape[0]
        group_add[level] = np.sum(group_add[level],axis=0)
        group_add[level] = np.array((group_add[level] - member_group*q) / (p-q))
        # group_add[level] = np.nan_to_num(group_add[level] / sum(group_add[level]))
        group_add[level] = np.round(NormSub(member_group,group_add[level]))
    return group_add

In [ ]:
def construct_HDG(k,data):
    new_data = np.random.randint(1, total_dim+comb(total_dim,2)+1, size=(total_n, 1))
    new_column = pd.Series(new_data.flatten())
    transdf["group"] = new_column
    attributes = ['attribute' + str(i) for i in range(total_dim)]
    true_one_grid = []
    one_grid = []
    one_group = []
    group_num = 1
    for i in range(k):
        c = 64
        group_data = data.loc[data['group']==group_num]
        g = choose_g('HDG','g1',total_dim)
        tmp_true,tmp_grid,tmp_group = OUE(group_data,attributes[i],c,True)
        true_one_grid.append(tmp_true)
        one_grid.append(tmp_grid)
        tmp_one_group = DR(tmp_group,True)
        one_group.append(tmp_one_group)
        print(group_num)
        group_num = group_num+1

    true_two_grid = []
    two_grid = []
    two_group = []
    for i in range(k):        
        c1 = 64
        for j in range(i+1,k):
            group_data = data.loc[data['group']==group_num]
            c2 = 64
            g2 = choose_g('HDG','g2',total_dim)
            tmp_true,tmp_grid,tmp_group = two_OUE(group_data,attributes[i],attributes[j],c1,c2,True)
            # slice_grid = []
            # for slice in range(0,len(tmp_grid),c2):
            #     sli_tmp_grid = tmp_grid[slice:slice+c2]
            #     slice_grid.append(sli_tmp_grid)
            # two_grid.append(slice_grid)
            true_two_grid.append(tmp_true)
            two_grid.append(tmp_grid)
            tmp_two_group = DR(tmp_group,True)
            two_group.append(tmp_two_group)
            print(group_num)
            group_num = group_num+1

    return true_one_grid,one_grid,one_group,true_two_grid,two_grid,two_group

In [ ]:
def construct_CORR(k,data):
    new_data = np.random.randint(1, total_dim+comb(total_dim,2)+1, size=(total_n, 1))
    new_column = pd.Series(new_data.flatten())
    transdf["group"] = new_column
    attributes = ['attribute' + str(i) for i in range(total_dim)]
    true_one_grid = []
    one_grid = []
    one_group = []
    group_num = 1
    for i in range(k):
        c = 64
        group_data = data.loc[data['group']==group_num]
        g = choose_g('HDG','g1',total_dim)
        tmp_true,tmp_grid,tmp_group = OUE(group_data,attributes[i],c,True)
        true_one_grid.append(tmp_true)
        one_grid.append(tmp_grid)
        tmp_one_group = DR(tmp_group,True)
        one_group.append(tmp_one_group)
        print(group_num)
        group_num = group_num+1
    correction_matrix = np.full((k,k),0,dtype=float)
    for i in range(k):        
        for j in range(i+1,k):
            one_gridi = copy.deepcopy(one_grid[i][epsilon_dim-1])/np.sum(one_grid[i][epsilon_dim-1])
            one_gridj = copy.deepcopy(one_grid[j][epsilon_dim-1])/np.sum(one_grid[i][epsilon_dim-1])
            pearson_corr = np.corrcoef(one_gridi, one_gridj)[0, 1]
            print(pearson_corr)
            if abs(pearson_corr) > 0.2:
                correction_matrix[i][j] = 1
    true_two_grid = []
    two_grid = []
    two_group = []
    need = np.sum(correction_matrix)
    group_num_range = range(group_num, int(total_dim+comb(total_dim,2)+1))
    remain_data = data.loc[data['group'].isin(group_num_range)]
    remain_data.reset_index(drop=True, inplace=True)  # 重新设置索引
    new_data1 = np.random.randint(group_num, group_num+need, size=(len(remain_data), 1))
    new_column1 = pd.Series(new_data1.flatten())
    remain_data['group'] = new_column1
    for i in range(k):        
        c1 = 64
        for j in range(i+1,k):
            c2 = 64
            if correction_matrix[i][j] == 1:
                group_data = remain_data.loc[remain_data['group']==group_num]
                tmp_true,tmp_grid,tmp_group = two_OUE(group_data,attributes[i],attributes[j],c1,c2,True)
                tmp_two_group = DR(tmp_group,True)
                for e in range(epsilon_dim):
                    tmp_true[e] = tmp_true[e]/np.sum(tmp_true[e])*member1[e]
                    tmp_grid[e] = tmp_grid[e]/np.sum(tmp_grid[e])*member1[e]
                    tmp_two_group[e] = tmp_two_group[e]/np.sum(tmp_two_group[e])*member[e]
                group_num = group_num+1
            else:
                tmp_grid = []
                tmp_two_group = []
                tmp_true = []
                g2 = choose_g('HDG','g2',total_dim)
                data1 = group_data.loc[group_data[epsilon_type]==j+1][[attributes[i]]].values
                data2 = group_data.loc[group_data[epsilon_type]==j+1][[attributes[j]]].values
                for e in range(epsilon_dim):
                    one_gridi = np.reshape(copy.deepcopy(one_grid[i][e])/np.sum(one_grid[i][e]), (1, g2[e], g[e]//g2[e]))
                    one_gridj = np.reshape(copy.deepcopy(one_grid[j][e])/np.sum(one_grid[i][e]), (1, g2[e], g[e]//g2[e]))
                    tmp_one_gridi = np.sum(one_gridi, axis=2)
                    tmp_one_gridj = np.sum(one_gridj, axis=2)
                    grid_res = np.matmul(tmp_one_gridi.reshape(g2[e], 1), tmp_one_gridj.reshape(1, g2[e])).flatten()
                    group_res = np.matmul(tmp_one_gridi.reshape(g2[e], 1), tmp_one_gridj.reshape(1, g2[e])).flatten()
                    tmp_grid.append(member1[e]*(grid_res/np.sum(grid_res)))
                    tmp_two_group.append(member[e]*(group_res/np.sum(group_res)))
                    block1 = np.round(c1/g2)
                    block2 = np.round(c2/g2)
                    input_data1 = data1 // block1[e]
                    input_data2 = data2 // block2[e]
                    k1 = math.ceil(c1/block1[e])
                    k2 = math.ceil(c2/block2[e])
                    true_freq_one = np.zeros(k1*k2,dtype=int)
                    input_ue_data = np.zeros((len(input_data1),k1*k2),dtype=int)
                    # UE perturbation function
                    for inx in range(len(data1)):
                        # Unary encoding
                        true_data = input_data1[inx]*k2 + input_data2[inx]
                        input_ue_data[inx][int(true_data)] = 1
                        true_freq_one += input_ue_data[inx]
                    true_freq_one = true_freq_one/np.sum(true_freq_one)*member1[e]
                    tmp_true.append(true_freq_one)
            # slice_grid = []
            # for slice in range(0,len(tmp_grid),c2):
            #     sli_tmp_grid = tmp_grid[slice:slice+c2]
            #     slice_grid.append(sli_tmp_grid)
            # two_grid.append(slice_grid)
            true_two_grid.append(tmp_true)
            two_grid.append(tmp_grid)
            two_group.append(tmp_two_group)
            print(i)

    return true_one_grid,one_grid,one_group,true_two_grid,two_grid,two_group,correction_matrix


In [ ]:
def construct_LHIO(k,data):
    new_data = np.random.randint(1, total_group_LHIO+1, size=(total_n, 1))
    new_column = pd.Series(new_data.flatten())
    transdf["group"] = new_column
    attributes = ['attribute' + str(i) for i in range(total_dim)]
    true_two_grid = []
    two_grid = []
    group_num = 1
    for i in range(k):        
        for j in range(i+1,k):
            true_two_grid_one = []
            two_grid_one = []
            for m in range(int(math.log(c_domain)/math.log(fan_out)+1)):
                for n in range(int(math.log(c_domain)/math.log(fan_out)+1)):
                    group_data = data.loc[data['group']==group_num]
                    g1 = int(c_domain//(fan_out**(math.log(c_domain)/math.log(fan_out)-m)))
                    g2 = int(c_domain//(fan_out**(math.log(c_domain)/math.log(fan_out)-n)))
                    tmp_true,tmp_grid = two_LHIO(group_data,attributes[i],attributes[j],g1,g2,True)
                    true_two_grid_one.append(tmp_true)
                    two_grid_one.append(tmp_grid)
                    group_num = group_num+1
            true_two_grid.append(true_two_grid_one)
            two_grid.append(two_grid_one)
            print(i)
    return true_two_grid,two_grid

In [ ]:
def consistency(one_grid,two_grid,str1):
    w = total_dim
    m = int(comb(w,2))
    member_cn = np.zeros(epsilon_dim,dtype=int)
    if str1 == 'ndr':
        member_cn = member1
    else:
        member_cn = member

    g1 = choose_g('HDG','g1',total_dim)
    g2 = choose_g('HDG','g2',total_dim)
    theta = np.zeros((epsilon_dim,w,w),dtype=float)
    for e in range(epsilon_dim):#epsilon level 
        for i in range(w):#attribute a
            for j in range(w):#w grids related to a
                if i == j:
                    theta[e][i][j] = 1/g1[e]
                else:
                    theta[e][i][j] = 1/g2[e]
            theta[e][i] = np.nan_to_num(theta[e][i] / sum(theta[e][i]))
            plus = g1[e]//g2[e]
            for cell in range(g2[e]):
                S = np.zeros((epsilon_dim,w,w),dtype=float)
                total_clo = np.zeros(w,dtype=float)
                for j in range(w):
                    if i == j:# one grid related to a
                        S[e][i][j] = S[e][i][j] + theta[e][i][j]*np.sum(one_grid[i][e][cell:cell+plus])
                    elif i>j: #two grids before a
                        for x in range(g2[e]):
                            S[e][i][j] = S[e][i][j] + theta[e][i][j]*two_grid[index[i][j]][e][cell+g2[e]*x]
                            total_clo[j] = total_clo[j] + two_grid[index[i][j]][e][cell+g2[e]*x]
                    elif i<j: #two grids after a
                        S[e][i][j] = S[e][i][j] + theta[e][i][j]*np.sum(two_grid[index[i][j]][e][cell*g2[e]:(cell+1)*g2[e]])
                total = sum(S[e][i])
                for j in range(w):
                    if i == j:
                        for x in range(plus):
                            one_grid[i][e][cell+x] = one_grid[i][e][cell+x] + (total-np.sum(one_grid[i][e][cell:cell+plus]))/plus
                    if i>j:
                        for x in range(g2[e]):
                            two_grid[index[j][i]][e][cell+g2[e]*x] = two_grid[index[j][i]][e][cell+g2[e]*x] + (total - total_clo[j])/g2[e]
                    elif i<j:
                        for x in range(g2[e]):
                            two_grid[index[i][j]][e][cell*g2[e]+x] = two_grid[index[i][j]][e][cell*g2[e]+x] + (total-sum(two_grid[index[i][j]][e][cell*g2[e]:(cell+1)*g2[e]]))/g2[e]

            one_grid[i][e] = NormSub(member_cn[e],one_grid[i][e])
    for i in range(m):
        for j in range(epsilon_dim):
            two_grid[i][j] = NormSub(member_cn[j],two_grid[i][j])
    return one_grid,two_grid

In [ ]:
#build respond matrix,only used for partial grids
def build_matrix(one_grid,two_grid,i,j,str1):
    matrix_final = []
    if str1 =='ndr':
        member_ba = member1
    elif str1 =='dr':
        member_ba = member
    g1 = choose_g('HDG','g1',total_dim)
    g2 = choose_g('HDG','g2',total_dim)
    attributes = ['attribute' + str(inx) for inx in range(total_dim)]
    c1 = c_domain
    c2 = c_domain
    for e in range(epsilon_dim):
        tmp_one_gridi = one_grid[i][e]
        tmp_one_gridj = one_grid[j][e]
        G = index[i][j]
        tmp_two_grid = two_grid[G][e]
        matrix = np.full((c1,c2),member_ba[e]/(c1*c2))
        iteration_max = 100
        for iterta in range(iteration_max):
            matrix_tmp = copy.deepcopy(matrix)
            length = c1//g1[e]
            for cell in range(len(tmp_one_gridi)):  #Gi
                s = 0
                s = s + np.sum(matrix[cell*length:cell*length+length,:])
                if s != 0:
                    matrix[cell*length:cell*length+length,:] = matrix[cell*length:cell*length+length,:]/s*tmp_one_gridi[cell]
                matrix = matrix/np.sum(matrix)*member_ba[e]
            width = c2//g1[e]
            for cell in range(len(tmp_one_gridj)):  #Gj
                s = 0
                s = s + np.sum(matrix[:,cell*width:cell*width+width])
                if s != 0:
                    matrix[:,cell*width:cell*width+width] = matrix[:,cell*width:cell*width+width]/s*tmp_one_gridj[cell]
                matrix = matrix/np.sum(matrix)*member_ba[e]
            length = c1//g2[e]
            width = c2//g2[e]
            for cell1 in range(g2[e]):  #Gij
                for cell2 in range(g2[e]):
                    s = 0
                    s = s + np.sum(matrix[cell1*length:cell1*length+length,cell2*width:cell2*width+width])
                    if s != 0:
                        matrix[cell1*length:cell1*length+length,cell2*width:cell2*width+width] = matrix[cell1*length:cell1*length+length,cell2*width:cell2*width+width]/s*tmp_two_grid[cell1*g2[e]+cell2]
                    matrix = matrix/np.sum(matrix)*member_ba[e]
            diff = np.sum(np.abs(matrix-matrix_tmp))
            if diff < 1:
                break
        matrix_trans = np.zeros((g2[e],g2[e]))
        for inxi in range(g2[e]):
            for inxj in range(g2[e]):
                matrix_trans[inxi][inxj] = np.sum(matrix[inxi*length:inxi*length+length,inxj*width:inxj*width+width])
        matrix_final.append(matrix_trans)
    return matrix_final

In [ ]:
def range_query_list(k):
    if k == 1:
        t1 = pd.read_table('query_1.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0'])
        t1.drop(labels=['c1','c2','c3'],axis=1,inplace=True)
        return t1
    if k == 2:
        t2 = pd.read_table('query_2.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1'])
        t2.drop(labels=['c1','c2','c3','c6'],axis=1,inplace=True)
        return t2
    if k == 3:
        t3 = pd.read_table('query_3.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2'])
        t3.drop(labels=['c1','c2','c3','c6','c7'],axis=1,inplace=True)
        return t3
    if k == 4:
        t4 = pd.read_table('query_4.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3'])
        t4.drop(labels=['c1','c2','c3','c6','c7','c8'],axis=1,inplace=True)
        return t4
    if k == 5:
        t5 = pd.read_table('query_5.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3','c9', 'attribute4','range4'])
        t5.drop(labels=['c1','c2','c3','c6','c7','c8','c9'],axis=1,inplace=True)
        return t5
    if k == 6:
        t6 = pd.read_table('query_6.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3','c9', 'attribute4','range4','c10', 'attribute5','range5'])
        t6.drop(labels=['c1','c2','c3','c6','c7','c8','c9','c10'],axis=1,inplace=True)
        return t6
    if k == 7:
        t7 = pd.read_table('query_7.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3','c9', 'attribute4','range4','c10', 'attribute5','range5','c11', 'attribute6','range6'])
        t7.drop(labels=['c1','c2','c3','c6','c7','c8','c9','c10','c11'],axis=1,inplace=True)
        return t7
    if k == 8:
        t8 = pd.read_table('query_8.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3','c9', 'attribute4','range4','c10', 'attribute5','range5','c11', 'attribute6','range6', 'c12', 'attribute7','range7'])
        t8.drop(labels=['c1','c2','c3','c6','c7','c8','c9','c10','c11','c12'],axis=1,inplace=True)
        return t8
    if k == 9:
        t9 = pd.read_table('query_9.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3','c9', 'attribute4','range4','c10', 'attribute5','range5','c11', 'attribute6','range6', 'c12', 'attribute7','range7','c13', 'attribute8','range8'])
        t9.drop(labels=['c1','c2','c3','c6','c7','c8','c9','c10','c11','c12','c13'],axis=1,inplace=True)
        return t9
    if k == 10:
        t10 = pd.read_table('query_10.txt', sep='\s|=',names=['c1','c2', 'c3','attribute0','range0','c6', 'attribute1','range1','c7', 'attribute2','range2','c8', 'attribute3','range3','c9', 'attribute4','range4','c10', 'attribute5','range5','c11', 'attribute6','range6', 'c12', 'attribute7','range7','c13', 'attribute8','range8','c14', 'attribute9','range9'])
        t10.drop(labels=['c1','c2','c3','c6','c7','c8','c9','c10','c11','c12','c13','c14'],axis=1,inplace=True)
        return t10

In [ ]:
def real_ans(lam,query_two_dim,data,epsilon):
    real_res = []
    for q in range(len(query_two_dim)):
        sum = np.zeros(epsilon_dim,dtype=int)
        range_dim = np.zeros((total_dim,2),dtype=int)
        for l in range(lam):
            range_tmp = str(query_two_dim['range'+str(l)][q]).split(',')
            range_tmp = list(map(int, range_tmp))
            if len(range_tmp) == 2:
                range_dim[l][0] = range_tmp[0]
                range_dim[l][1] = range_tmp[1]
            else:
                range_dim[l][0] = range_tmp[0]
                range_dim[l][1] = range_tmp[0]
        if lam == 1:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 2:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 3:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 4:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 5:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[query_two_dim['attribute4'][q]] >= range_dim[4][0]) & (transdf[query_two_dim['attribute4'][q]] <= range_dim[4][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 6:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[query_two_dim['attribute4'][q]] >= range_dim[4][0]) & (transdf[query_two_dim['attribute4'][q]] <= range_dim[4][1]) & 
                                   (transdf[query_two_dim['attribute5'][q]] >= range_dim[5][0]) & (transdf[query_two_dim['attribute5'][q]] <= range_dim[5][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 7:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[query_two_dim['attribute4'][q]] >= range_dim[4][0]) & (transdf[query_two_dim['attribute4'][q]] <= range_dim[4][1]) & 
                                   (transdf[query_two_dim['attribute5'][q]] >= range_dim[5][0]) & (transdf[query_two_dim['attribute5'][q]] <= range_dim[5][1]) & 
                                   (transdf[query_two_dim['attribute6'][q]] >= range_dim[6][0]) & (transdf[query_two_dim['attribute6'][q]] <= range_dim[6][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 8:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[query_two_dim['attribute4'][q]] >= range_dim[4][0]) & (transdf[query_two_dim['attribute4'][q]] <= range_dim[4][1]) & 
                                   (transdf[query_two_dim['attribute5'][q]] >= range_dim[5][0]) & (transdf[query_two_dim['attribute5'][q]] <= range_dim[5][1]) & 
                                   (transdf[query_two_dim['attribute6'][q]] >= range_dim[6][0]) & (transdf[query_two_dim['attribute6'][q]] <= range_dim[6][1]) & 
                                   (transdf[query_two_dim['attribute7'][q]] >= range_dim[7][0]) & (transdf[query_two_dim['attribute7'][q]] <= range_dim[7][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 9:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[query_two_dim['attribute4'][q]] >= range_dim[4][0]) & (transdf[query_two_dim['attribute4'][q]] <= range_dim[4][1]) & 
                                   (transdf[query_two_dim['attribute5'][q]] >= range_dim[5][0]) & (transdf[query_two_dim['attribute5'][q]] <= range_dim[5][1]) & 
                                   (transdf[query_two_dim['attribute6'][q]] >= range_dim[6][0]) & (transdf[query_two_dim['attribute6'][q]] <= range_dim[6][1]) & 
                                   (transdf[query_two_dim['attribute7'][q]] >= range_dim[7][0]) & (transdf[query_two_dim['attribute7'][q]] <= range_dim[7][1]) & 
                                   (transdf[query_two_dim['attribute8'][q]] >= range_dim[8][0]) & (transdf[query_two_dim['attribute8'][q]] <= range_dim[8][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        if lam == 10:
            for e in range(epsilon_dim):
                data = transdf.loc[(transdf[query_two_dim['attribute0'][q]] >= range_dim[0][0]) & (transdf[query_two_dim['attribute0'][q]] <= range_dim[0][1]) & 
                                   (transdf[query_two_dim['attribute1'][q]] >= range_dim[1][0]) & (transdf[query_two_dim['attribute1'][q]] <= range_dim[1][1]) & 
                                   (transdf[query_two_dim['attribute2'][q]] >= range_dim[2][0]) & (transdf[query_two_dim['attribute2'][q]] <= range_dim[2][1]) & 
                                   (transdf[query_two_dim['attribute3'][q]] >= range_dim[3][0]) & (transdf[query_two_dim['attribute3'][q]] <= range_dim[3][1]) & 
                                   (transdf[query_two_dim['attribute4'][q]] >= range_dim[4][0]) & (transdf[query_two_dim['attribute4'][q]] <= range_dim[4][1]) & 
                                   (transdf[query_two_dim['attribute5'][q]] >= range_dim[5][0]) & (transdf[query_two_dim['attribute5'][q]] <= range_dim[5][1]) & 
                                   (transdf[query_two_dim['attribute6'][q]] >= range_dim[6][0]) & (transdf[query_two_dim['attribute6'][q]] <= range_dim[6][1]) & 
                                   (transdf[query_two_dim['attribute7'][q]] >= range_dim[7][0]) & (transdf[query_two_dim['attribute7'][q]] <= range_dim[7][1]) & 
                                   (transdf[query_two_dim['attribute8'][q]] >= range_dim[8][0]) & (transdf[query_two_dim['attribute8'][q]] <= range_dim[8][1]) & 
                                   (transdf[query_two_dim['attribute9'][q]] >= range_dim[9][0]) & (transdf[query_two_dim['attribute9'][q]] <= range_dim[9][1]) & 
                                   (transdf[epsilon_type] == (e + 1))]
                sum[e] = len(data)
        real_res.append(sum)
    print(real_res)
    return real_res

In [ ]:
def range_query_two_HDG(dim,one_grid,two_grid,str1):

    attributes = ['attribute' + str(i) for i in range(total_dim)]
    one_range_res = []
    two_range_res = []
    if str1 == 'ndr':
        member_cn = member1
    else:
        member_cn = member
    for q in range(len(dim)):
        #ONE_ANS
        for i in range(len(dim.columns)//2):
            a = attributes.index(dim['attribute'+str(i)][q])
            range1 = str(dim['range'+str(i)][q]).split(',')
            range1 = list(map(int, range1))
            g1 = choose_g('HDG','g1',total_dim)
            res = np.zeros(epsilon_dim,dtype=float)
            for e in range(epsilon_dim):
                sum = 0
                length2 = g1[e]
                if len(range1) == 1:
                    res[e] = one_grid[a][e][int(range1[0])]
                else:
                    c = c_domain
                    block = choose_g('HDG','g1',total_dim)
                    block = np.round(c/block)
                    long1 = range1[1]-range1[0]+1
                    if c%block[e] != 0 and range1[1]//block[e] == g1[e]-1:
                        last = c%block[e]
                    else:
                        last = block[e]
                    if range1[1]//block[e] - range1[0]//block[e] == 0:
                        sum = sum + long1/block[e]*one_grid[a][e][int(range1[0]//block[e])]
                    else:
                        left  = range1[0]//block[e]
                        right = range1[1]//block[e]
                        num_grid = int(right - left + 1)
                        sum = sum + ((left+1)*block[e]-range1[0])/block[e]*one_grid[a][e][int(left)]
                        if num_grid > 2:
                            for inx in range(1,num_grid-1):
                                sum = sum + one_grid[a][e][int((left+inx))]
                        sum = sum + (range1[1]+1-right*block[e])/min(last,block[e])*one_grid[a][e][int(right)]
                    res[e] = sum
            one_range_res.append(res)
        #TWO_ANS
        for i in range(len(dim.columns)//2):
            for j in range(i+1,len(dim.columns)//2):
                a = attributes.index(dim['attribute'+str(i)][q])
                b = attributes.index(dim['attribute'+str(j)][q])
                G = index[a][b]
                Mij = build_matrix(one_grid,two_grid,a,b,str1)
                range1 = str(dim['range'+str(i)][q]).split(',')
                range2 = str(dim['range'+str(j)][q]).split(',')
                range1 = list(map(int, range1))
                range2 = list(map(int, range2))
                g2 = choose_g('HDG','g2',total_dim)
                #HDG
                res = np.zeros(epsilon_dim,dtype=float)
                for e in range(epsilon_dim):
                    sum = 0
                    length2 = g2[e]
                    if len(range1) == 1 and len(range2) == 1:#ord+ord
                        res[e] = two_grid[G][e][int(range1[0]*length2+range2[0])]
                    elif len(range1) == 2 and len(range2) == 1:#num+ord
                        c = c_domain
                        block = choose_g('HDG','g2',total_dim)
                        block = np.round(c/block)
                        long1 = range1[1]-range1[0]+1
                        if c%block[e] != 0 and range1[1]//block[e] == g2[e]-1:
                            last = c%block[e]
                        else:
                            last = block[e]
                        if range1[1]//block[e] - range1[0]//block[e] == 0:
                            sum = sum + long1/block[e]*Mij[e][int(range1[0]//block[e])][range2[0]]
                        else:
                            left  = int(range1[0]//block[e])
                            right = int(range1[1]//block[e])
                            num_grid = int(right - left + 1)
                            sum = sum + ((left+1)*block[e]-range1[0])/block[e]*Mij[e][int(left)][range2[0]]
                            if num_grid > 2:
                                for inx in range(1,num_grid-1):
                                    sum = sum + two_grid[G][e][int((left+inx)*length2+range2[0])]
                            sum = sum + (range1[1]+1-right*block[e])/min(last,block[e])*Mij[e][int(right)][range2[0]]
                        res[e] = sum
                    elif len(range1) == 1 and len(range2) == 2:#ord+num
                        c = c_domain
                        block = choose_g('HDG','g2',total_dim)
                        block = np.round(c/block)
                        long2 = range2[1]-range2[0]+1
                        if c%block[e] != 0 and range2[1]//block[e] == g2[e]-1:
                            last = c%block[e]
                        else:
                            last = block[e]
                        if range2[1]//block[e] - range2[0]//block[e] == 0:
                            sum = sum + long2/block[e]*Mij[e][range1[0]][int(range2[1]//block[e])]
                        else:
                            left  = range2[0]//block[e]
                            right = range2[1]//block[e]
                            num_grid = int(right - left + 1)
                            sum = sum + ((left+1)*block[e]-range2[0])/block[e]*Mij[e][range1[0]][int(left)]
                            if num_grid > 2:
                                for inx in range(1,num_grid-1):
                                    sum = sum + two_grid[G][e][int(range1[0]*length2+left+inx)]
                            sum = sum + (range2[1]+1-right*block[e])/min(last,block[e])*Mij[e][range1[0]][int(right)]
                        res[e] = sum
                    else:#num+num
                        c1 = c_domain
                        c2 = c_domain
                        block1 = choose_g('HDG','g2',total_dim)
                        block1 = np.round(c1/block1)
                        block2 = choose_g('HDG','g2',total_dim)
                        block2 = np.round(c2/block2)
                        left1  = int(range1[0]//block1[e])
                        right1 = int(range1[1]//block1[e])
                        left2  = int(range2[0]//block2[e])
                        right2 = int(range2[1]//block2[e])
                        num_grid1 = int(right1 - left1 + 1)
                        num_grid2 = int(right2 - left2 + 1)
                        long1 = range1[1]-range1[0]+1
                        long2 = range2[1]-range2[0]+1
                        if c1%block1[e] != 0 and range1[1]//block1[e] == g2[e]-1:
                            last1 = c1%block1[e]
                        else:
                            last1 = block1[e]
                        if c2%block2[e] != 0 and range2[1]//block2[e] == g2[e]-1:
                            last2 = c2%block2[e]
                        else:
                            last2 = block2[e]
                        if left1 == right1 and left2 == right2:
                            sum = sum + long1*long2/(block1[e]*block2[e])*Mij[e][left1][left2]
                        elif left1 == right1 and left2 != right2:
                            sum = sum + long1*((left2+1)*block2[e]-range2[0])/(block1[e]*block2[e])*Mij[e][left1][left2]
                            if num_grid2 > 2:
                                for inx in range(1,num_grid2-1):
                                    sum = sum + long1/block1[e]*Mij[e][left1][left2+inx]
                            sum = sum + long1*(range2[1]+1-right2*block2[e])/(block1[e]*min(last2,block2[e]))*Mij[e][left1][right2]
                        elif left1 != right1 and left2 == right2:
                            sum = sum + ((left1+1)*block1[e]-range1[0])*long2/(block1[e]*block2[e])*Mij[e][left1][left2]
                            if num_grid1 > 2:
                                for inx in range(1,num_grid1-1):
                                    sum = sum + long2/block2[e]*Mij[e][left1+inx][left2]
                            sum = sum + (range1[1]+1-right1*block1[e])*long2/(min(last1,block1[e])*block2[e])*Mij[e][right1][left2]
                        else:
                            sum = sum + ((left1+1)*block1[e]-range1[0])*((left2+1)*block2[e]-range2[0])/(block1[e]*block2[e])*Mij[e][left1][left2]
                            if num_grid1 > 2:
                                for inx in range(1,num_grid1-1):
                                    sum = sum + ((left2+1)*block2[e]-range2[0])/block2[e]*Mij[e][left1+inx][left2]
                                    sum = sum + (range2[1]+1-right2*block2[e])/min(last2,block2[e])*Mij[e][left1+inx][right2]
                            sum = sum + ((left1+1)*block1[e]-range1[0])*(range2[1]+1-right2*block2[e])/(block1[e]*min(last2,block2[e]))*Mij[e][left1][right2]
                            if num_grid2 > 2:
                                for inx in range(1,num_grid2-1):
                                    sum = sum + ((left1+1)*block1[e]-range1[0])/block1[e]*Mij[e][left1][left2+inx]
                                    sum = sum + (range1[1]+1-right1*block1[e])/min(last1,block1[e])*Mij[e][right1][left2+inx]
                            sum = sum + (range1[1]+1-right1*block1[e])*((left2+1)*block2[e]-range2[0])/(min(last1,block1[e])*block2[e])*Mij[e][right1][left2]
                            sum = sum + (range1[1]+1-right1*block1[e])*(range2[1]+1-right2*block2[e])/(min(last1,block1[e])*min(last2,block2[e]))*Mij[e][right1][right2]
                            if num_grid1>2 and num_grid2>2:
                                for inx1 in range(1,num_grid1-1):
                                    for inx2 in range(1,num_grid2-1):
                                        sum = sum + Mij[e][int(left1+inx1)][int(left2+inx2)]
                        res[e] = sum
                two_range_res.append(res)
    return one_range_res,two_range_res

In [ ]:
def range_query_two_TDG(dim,one_grid,two_grid,str1):

    attributes = ['attribute' + str(i) for i in range(total_dim)]
    one_range_res = []
    two_range_res = []
    if str1 == 'ndr':
        member_cn = member1
    else:
        member_cn = member
    for q in range(len(dim)):
        #ONE_ANS
        for i in range(len(dim.columns)//2):
            a = attributes.index(dim['attribute'+str(i)][q])
            range1 = str(dim['range'+str(i)][q]).split(',')
            range1 = list(map(int, range1))
            g1 = choose_g('HDG','g1',total_dim)
            res = np.zeros(epsilon_dim,dtype=float)
            for e in range(epsilon_dim):
                sum = 0
                length2 = g1[e]
                if len(range1) == 1:
                    res[e] = one_grid[a][e][int(range1[0])]
                else:
                    c = len(set(transdf[dim['attribute'+str(i)][q]]))
                    block = choose_g('HDG','g1',total_dim)
                    block = np.round(c/block)
                    long1 = range1[1]-range1[0]+1
                    if c%block[e] != 0 and range1[1]//block[e] == g1[e]-1:
                        last = c%block[e]
                    else:
                        last = block[e]
                    if range1[1]//block[e] - range1[0]//block[e] == 0:
                        sum = sum + long1/block[e]*one_grid[a][e][int(range1[0]//block[e])]
                    else:
                        left  = range1[0]//block[e]
                        right = range1[1]//block[e]
                        num_grid = int(right - left + 1)
                        sum = sum + ((left+1)*block[e]-range1[0])/block[e]*one_grid[a][e][int(left)]
                        if num_grid > 2:
                            for inx in range(1,num_grid-1):
                                sum = sum + one_grid[a][e][int((left+inx))]
                        sum = sum + (range1[1]+1-right*block[e])/min(last,block[e])*one_grid[a][e][int(right)]
                    res[e] = sum
            one_range_res.append(res)
        #TWO_ANS
        for i in range(len(dim.columns)//2):
            for j in range(i+1,len(dim.columns)//2):
                a = attributes.index(dim['attribute'+str(i)][q])
                b = attributes.index(dim['attribute'+str(j)][q])
                G = index[a][b]
                range1 = str(dim['range'+str(i)][q]).split(',')
                range2 = str(dim['range'+str(j)][q]).split(',')
                range1 = list(map(int, range1))
                range2 = list(map(int, range2))
                g2 = choose_g('HDG','g2',total_dim)
                res = np.zeros(epsilon_dim,dtype=float)
                for e in range(epsilon_dim):
                    sum = 0
                    length2 = g2[e]
                    if len(range1) == 1 and len(range2) == 1:#ord+ord
                        res[e] = two_grid[G][e][int(range1[0]*length2+range2[0])]
                    elif len(range1) == 2 and len(range2) == 1:#num+ord
                        c = len(set(transdf[dim['attribute'+str(i)][q]]))
                        block = choose_g('TDG','g2',total_dim)
                        block = np.round(c/block)
                        long1 = range1[1]-range1[0]+1
                        if c%block[e] != 0 and range1[1]//block[e] == g2[e]-1:
                            last = c%block[e]
                        else:
                            last = block[e]
                        if range1[1]//block[e] - range1[0]//block[e] == 0:
                            sum = sum + long1/block[e]*two_grid[G][e][int(range1[0]//block[e])*length2+range2[0]]
                        else:
                            left  = range1[0]//block[e]
                            right = range1[1]//block[e]
                            num_grid = int(right - left + 1)
                            sum = sum + ((left+1)*block[e]-range1[0])/block[e]*two_grid[G][e][int(left*length2+range2[0])]
                            if num_grid > 2:
                                for inx in range(1,num_grid-1):
                                    sum = sum + two_grid[G][e][int((left+inx)*length2+range2[0])]
                            sum = sum + (range1[1]+1-right*block[e])/min(last,block[e])*two_grid[G][e][int(right*length2+range2[0])]
                        res[e] = sum
                    elif len(range1) == 1 and len(range2) == 2:#ord+num
                        c = len(set(transdf[dim['attribute'+str(j)][q]]))
                        block = choose_g('TDG','g2',64)
                        block = np.round(c/block)
                        long2 = range2[1]-range2[0]+1
                        if c%block[e] != 0 and range2[1]//block[e] == g2[e]-1:
                            last = c%block[e]
                        else:
                            last = block[e]
                        if range2[1]//block[e] - range2[0]//block[e] == 0:
                            sum = sum + long2/block[e]*two_grid[G][e][int(range1[0]*length2+range2[1]//block[e])]
                        else:
                            left  = range2[0]//block[e]
                            right = range2[1]//block[e]
                            num_grid = int(right - left + 1)
                            sum = sum + ((left+1)*block[e]-range2[0])/block[e]*two_grid[G][e][int(range1[0]*length2+left)]
                            if num_grid > 2:
                                for inx in range(1,num_grid-1):
                                    sum = sum + two_grid[G][e][int(range1[0]*length2+left+inx)]
                            sum = sum + (range2[1]+1-right*block[e])/min(last,block[e])*two_grid[G][e][int(range1[0]*length2+right)]
                        res[e] = sum
                    else:#num+num
                        c1 = len(set(transdf[dim['attribute'+str(i)][q]]))
                        c2 = len(set(transdf[dim['attribute'+str(j)][q]]))
                        block1 = choose_g('TDG','g2',total_dim)
                        block1 = np.round(c1/block1)
                        block2 = choose_g('TDG','g2',total_dim)
                        block2 = np.round(c2/block2)
                        left1  = range1[0]//block1[e]
                        right1 = range1[1]//block1[e]
                        left2  = range2[0]//block2[e]
                        right2 = range2[1]//block2[e]
                        num_grid1 = int(right1 - left1 + 1)
                        num_grid2 = int(right2 - left2 + 1)
                        long1 = range1[1]-range1[0]+1
                        long2 = range2[1]-range2[0]+1
                        if c1%block1[e] != 0 and range1[1]//block1[e] == g2[e]-1:
                            last1 = c1%block1[e]
                        else:
                            last1 = block1[e]
                        if c2%block2[e] != 0 and range2[1]//block2[e] == g2[e]-1:
                            last2 = c2%block2[e]
                        else:
                            last2 = block2[e]
                        if left1 == right1 and left2 == right2:
                            sum = sum + long1*long2/(block1[e]*block2[e])*two_grid[G][e][int(left1*length2+left2)]
                        elif left1 == right1 and left2 != right2:
                            sum = sum + long1*((left2+1)*block2[e]-range2[0])/(block1[e]*block2[e])*two_grid[G][e][int(left1*length2+left2)]
                            if num_grid2 > 2:
                                for inx in range(1,num_grid2-1):
                                    sum = sum + long1/block1[e]*two_grid[G][e][int(left1*length2+left2+inx)]
                            sum = sum + long1*(range2[1]+1-right2*block2[e])/(block1[e]*min(last2,block2[e]))*two_grid[G][e][int(left1*length2+right2)]
                        elif left1 != right1 and left2 == right2:
                            sum = sum + ((left1+1)*block1[e]-range1[0])*long2/(block1[e]*block2[e])*two_grid[G][e][int(left1*length2+left2)]
                            if num_grid1 > 2:
                                for inx in range(1,num_grid1-1):
                                    sum = sum + long2/block2[e]*two_grid[G][e][int((left1+inx)*length2+left2)]
                            sum = sum + (range1[1]+1-right1*block1[e])*long2/(min(last1,block1[e])*block2[e])*two_grid[G][e][int(right1*length2+left2)]
                        else:
                            sum = sum + ((left1+1)*block1[e]-range1[0])*((left2+1)*block2[e]-range2[0])/(block1[e]*block2[e])*two_grid[G][e][int(left1*length2+left2)]
                            if num_grid1 > 2:
                                for inx in range(1,num_grid1-1):
                                    sum = sum + ((left2+1)*block2[e]-range2[0])/block2[e]*two_grid[G][e][int((left1+inx)*length2+left2)]
                                    sum = sum + (range2[1]+1-right2*block2[e])/min(last2,block2[e])*two_grid[G][e][int((left1+inx)*length2+right2)]
                            sum = sum + ((left1+1)*block1[e]-range1[0])*(range2[1]+1-right2*block2[e])/(block1[e]*min(last2,block2[e]))*two_grid[G][e][int(left1*length2+right2)]
                            if num_grid2 > 2:
                                for inx in range(1,num_grid2-1):
                                    sum = sum + ((left1+1)*block1[e]-range1[0])/block1[e]*two_grid[G][e][int(left1*length2+left2+inx)]
                                    sum = sum + (range1[1]+1-right1*block1[e])/min(last1,block1[e])*two_grid[G][e][int(right1*length2+left2+inx)]
                            sum = sum + (range1[1]+1-right1*block1[e])*((left2+1)*block2[e]-range2[0])/(min(last1,block1[e])*block2[e])*two_grid[G][e][int(right1*length2+left2)]
                            sum = sum + (range1[1]+1-right1*block1[e])*(range2[1]+1-right2*block2[e])/(min(last1,block1[e])*min(last2,block2[e]))*two_grid[G][e][int(right1*length2+right2)]
                            if num_grid1>2 and num_grid2>2:
                                for inx1 in range(1,num_grid1-1):
                                    for inx2 in range(1,num_grid2-1):
                                        sum = sum + two_grid[G][e][int((left1+inx1)*length2+left2+inx2)]
                        res[e] = sum
                two_range_res.append(res)
    return one_range_res,two_range_res

In [ ]:
def find_containing_ranges(tree, target_range):
    result = []
    noede_index = []
    remaining_range = [copy.deepcopy(target_range),copy.deepcopy(target_range)]
    merged_range = [c_domain, 0]
    for level in tree:
        for node_range in level:
            if (node_range[0] >= remaining_range[0][0] and node_range[1] <= remaining_range[0][1]) or (node_range[0] >= remaining_range[1][0] and node_range[1] <= remaining_range[1][1]):
                result.append(node_range)
                for level_inx, sublist in enumerate(tree):
                    for item in sublist:
                        if item == node_range:
                            noede_index.append([level_inx,sublist.index(item)])
                            break
                # noede_index.append([tree.index(level)][level.index(node_range)])
                merged_range[0] = min(merged_range[0],node_range[0])
                merged_range[1] = max(merged_range[1],node_range[1])
        if level == tree[0]:
            continue
        else:
            remaining_range[0][0] = target_range[0]
            remaining_range[0][1] = merged_range[0]-1
            remaining_range[1][0] = merged_range[1]+1
            remaining_range[1][1] = target_range[1]
    
    return result,noede_index

def range_query_two_LHIO(dim,two_grid):

    attributes = ['attribute' + str(i) for i in range(total_dim)]
    two_range_res = []
    for q in range(len(dim)):
        #TWO_ANS
        for i in range(len(dim.columns)//2):
            for j in range(i+1,len(dim.columns)//2):
                a = attributes.index(dim['attribute'+str(i)][q])
                b = attributes.index(dim['attribute'+str(j)][q])
                G = index[a][b]
                range1 = str(dim['range'+str(i)][q]).split(',')
                range2 = str(dim['range'+str(j)][q]).split(',')
                range1 = list(map(int, range1))
                range2 = list(map(int, range2))
                attr_grid = two_grid[G]
                res = np.zeros(epsilon_dim,dtype=float)
                tree_high = int(math.log(c_domain)/math.log(fan_out)+1)
                tree = [[0, c_domain-1]]
                level_tree = []
                for inx in range(tree_high):
                    for sublist in tree:
                        level = []
                        start, end = sublist
                        step = (end - start + 1) // (fan_out**inx)
                        for inx_step in range(fan_out**inx):
                            new_start = start + inx_step * step
                            new_end = new_start + step - 1
                            level.append([new_start, new_end])
                    level_tree.append(level)
                nodes1,nodes_inx1 = find_containing_ranges(level_tree, range1)
                nodes2,nodes_inx2 = find_containing_ranges(level_tree, range2)

                for e in range(epsilon_dim):
                    sum = 0
                    if len(range1) == 1 and len(range2) == 1:#ord+ord
                        tree_level = int(math.log(c_domain)/math.log(fan_out))
                        res[e] = attr_grid[tree_level][e][int(range1[0]*c_domain+range2[0])]
                    elif len(range1) == 2 and len(range2) == 1:#num+ord
                        tree_level2 = int(math.log(c_domain)/math.log(fan_out))
                        tree_level1 = [x[0]*tree_high+tree_level2 for x in nodes_inx1]
                        nodes_result = [x[1]*(fan_out**tree_level2)+range2[0] for x in nodes_inx1]
                        for inx in range(len(tree_level1)):
                            sum = sum + attr_grid[tree_level1[inx]][e][nodes_result[inx]]
                        res[e] = sum
                    elif len(range1) == 1 and len(range2) == 2:#ord+num
                        tree_level1 = int(math.log(c_domain)/math.log(fan_out))
                        tree_level2 = [tree_level1*tree_high+x[0] for x in nodes_inx2]
                        nodes_result = [range1[0]*(fan_out**x[0])+x[1] for x in nodes_inx2]
                        for inx in range(len(tree_level2)):
                            sum = sum + attr_grid[tree_level2[inx]][e][nodes_result[inx]]
                        res[e] = sum
                    else:#num+num
                        tree_level = []
                        nodes_result = []
                        for x in nodes_inx1:
                            for y in nodes_inx2:
                                tree_level.append(x[0]*tree_high+y[0])
                                nodes_result.append(x[1]*(fan_out**y[0])+y[1])
                        for inx in range(len(tree_level)):
                            sum = sum + attr_grid[tree_level[inx]][e][nodes_result[inx]]
                        res[e] = sum
                    
                two_range_res.append(res)
                   
    return two_range_res

In [ ]:
def AC(level,query_list,two_grid,query_res,str1,str2):
    omega = np.zeros((len(query_list),epsilon_dim),dtype=float)
    attributes = ['attribute' + str(i) for i in range(total_dim)]

    if str2 =='lhio':
        member_cn = member_LHIO
    else:
        member_cn = member1
    gm = choose_g('HDG','g1',total_dim)
    B = np.zeros((len(query_list),epsilon_dim),dtype=float)
    for B_inx in range(len(query_list)):
        for i in range(len(query_list.columns)//2):
            for j in range(1,len(query_list.columns)//2):
                a = attributes.index(query_list['attribute'+str(i)][B_inx])
                b = attributes.index(query_list['attribute'+str(j)][B_inx])
                G = index[a][b]
                if str2 == 'lhio':
                    tmp_grid = copy.deepcopy(two_grid[G][int(math.log(c_domain)/math.log(fan_out)+1)**2-1])
                else:
                    tmp_grid = copy.deepcopy(two_grid[G])
                for e in range(level):
                    B_sum = 0
                    tmp_grid[e] = tmp_grid[e]/sum(tmp_grid[e])
                    for cell in range(len(tmp_grid[e])):
                        B_sum = B_sum + tmp_grid[e][cell]*tmp_grid[e][cell]
                    B[B_inx][e] = B[B_inx][e] + 1-B_sum
        B[B_inx] = B[B_inx]/comb(lamda_dim,2)
    if str1 == 'weighted':
        for q in range(len(query_list)):
            for e in range(level):
                omega[q][e] = 1/(((total_dim+comb(total_dim,2))/member_cn[e])*(4*gm[e]*np.exp((e+1)*l)/((np.exp((e+1)*l)-1)*(np.exp((e+1)*l)-1))+B[q][e]))
            omega[q] = omega[q]/sum(omega[q])
    if str1 == 'average':
        for q in range(len(query_list)):
            for e in range(level):
                omega[q][e] = member_cn[e]
            omega[q] = omega[q]/sum(omega[q])
    print(omega)
    res = np.zeros(len(query_list),dtype=float)
    if str1 == 'weighted' and str2 =='ndr':
        for q in range(len(query_list)):
            for e in range(level):
                res[q] = res[q] + query_res[q][e]*omega[q][e]/(member1[e]/np.sum(member1))*(total_dim+comb(total_dim,2))
    elif str1 == 'weighted' and str2 =='dr':
        for q in range(len(query_list)):
            for e in range(level):
                res[q] = res[q] + query_res[q][e]*omega[q][e]/(member[e]/np.sum(member1))*(total_dim+comb(total_dim,2))
    elif str1 == 'weighted' and str2 =='lhio':
        for q in range(len(query_list)):
            for e in range(level):
                res[q] = res[q] + query_res[q][e]*omega[q][e]/(member_LHIO[e]/np.sum(member_LHIO))*total_group_LHIO
    elif str1 == 'average' and str2 =='ndr':
        for q in range(len(query_list)):
            for e in range(level):
                res[q] = res[q] + query_res[q][e]*omega[q][e]/(member1[e]/np.sum(member1))*(total_dim+comb(total_dim,2))
    elif str1 == 'average' and str2 =='dr':
        for q in range(len(query_list)):
            for e in range(level):
                res[q] = res[q] + query_res[q][e]*omega[q][e]/(member[e]/np.sum(member1))*(total_dim+comb(total_dim,2))
    elif str1 == 'average' and str2 =='lhio':
        for q in range(len(query_list)):
            for e in range(level):
                res[q] = res[q] + query_res[q][e]*omega[q][e]/(member_LHIO[e]/np.sum(member_LHIO))*total_group_LHIO
    return res

In [ ]:
class cartesian(object):
    def __init__(self):
        self._data_list=[]

    def add_data(self,data=[]): #添加生成笛卡尔积的数据列表
        self._data_list.append(data)

    def build(self): #计算笛卡尔积
        #print(self._data_list)
        for item in itertools.product(*self._data_list):
            print(item)

def range_query_lamda_MEOM1(correction_matrix,lamda,query_lamda_dim,one_ans,lamda_ans,str1):
    attributes = ['attribute' + str(i) for i in range(total_dim)]
    corr_matrix = np.zeros((len(query_lamda_dim),int(comb(lamda,2))),dtype=int)
    for q in range(len(query_lamda_dim)):
        inx = 0
        for i in range(len(query_lamda_dim.columns)//2):
            for j in range(i+1,len(query_lamda_dim.columns)//2):
                a = attributes.index(query_lamda_dim['attribute'+str(i)][q])
                b = attributes.index(query_lamda_dim['attribute'+str(j)][q])
                corr_matrix[q][inx] = correction_matrix[a][b]
                inx = inx + 1
    if str1 =='ndr':
        member_cn = member1
    elif str1 =='dr':
        member_cn = member
    elif str1 =='lhio':
        member_cn = member_LHIO
    col = 2**lamda  # 计算矩阵的行数
    domain = []
    for i in range(lamda):
        domain.append(2)
    A1 = []
    for i in range(len(domain)):
        for j in range(domain[i]):
            a = np.zeros(domain, dtype=float)
            s = cartesian()
            for k in domain:
                s.add_data(range(0, k))
            for k in itertools.product(*s._data_list):
                if k[i] == j:
                    a[k] = 1
            A1.append(a.flatten())
    A1 = np.asarray(A1)
    # A1 = np.asarray(A1[::2])
    A1 = matrix(A1)

    A3 = matrix(1, (1, col), 'd')
    b3 = matrix([1.0])

    ans = np.zeros((len(query_lamda_dim),epsilon_dim),dtype=float)
    for q in range(len(query_lamda_dim)):
        for e in range(epsilon_dim):
            # b1 = []
            # for n in range(lamda):
            #     b1.append((one_ans[q*lamda+n//2][e])/member_cn[e])
            b1 = []
            for n in range(2*lamda):
                if n%2 == 0:
                    b1.append((one_ans[q*lamda+n//2][e])/member_cn[e])
                else:
                    b1.append((member_cn[e]-one_ans[q*lamda+n//2][e])/member_cn[e])
            b1 = matrix(b1)

            A2 = []
            b2 = []
            inx = 0
            for i in range(len(domain)):
                for m in range(i+1,len(domain)):
                    for j in range(domain[i]):
                        a = np.zeros(domain, dtype=float)
                        s = cartesian()
                        for k in domain:
                            s.add_data(range(0, k))
                        for k in itertools.product(*s._data_list):
                            if k[i] == j and k[m] == j:
                                a[k] = 1
                        if j%2 == 0 and corr_matrix[q][inx] == 1:
                            A2.append(a.flatten())
                            b2.append((lamda_ans[q*lamda+inx][e])/member_cn[e])
                    inx = inx + 1
            A2 = np.asarray(A2)
            A2 = matrix(A2)
            b2 = matrix(b2)
            if np.sum(corr_matrix[q]) == 0:
                A = matrix([A1,A3])
                b = matrix([b1,b3])
            else:
                A = matrix([A1,A2,A3])
                b = matrix([b1,b2,b3])
            # def F(x=None, z=None):
            #     if x is None: return 0, matrix(1.0, (col,1))
            #     if min(x) <= 0.0: return None
            #     f = -sum(x.T*log(x))
            #     Df = -(x**-1).T
            #     if z is None: return f, Df
            #     H = spdiag(z[0] * x**-2)
            #     return f, Df, H
            def F(x=None, z=None):
                # print(x)
                if x is None: return 0, matrix(1.0, (col, 1))
                # implicit constraint that x should be non-negative
                if min(x) <= 0: return None
                f = x.T * log(x)
                grad = 1.0 + log(x)
                if z is None: return f, grad.T
                H = spdiag(z[0] * x ** -1)
                return f, grad.T, H
            sol = solvers.cp(F, G=A, h=b, A=A3, b=b3)
            # sol = solvers.cp(F, G=A, h=b)
            p = sol['x']
            print(p)
            p = p*member_cn[e]
            ans[q][e] = p[0]
    return ans

In [ ]:
class cartesian(object):
    def __init__(self):
        self._data_list=[]

    def add_data(self,data=[]): #添加生成笛卡尔积的数据列表
        self._data_list.append(data)

    def build(self): #计算笛卡尔积
        #print(self._data_list)
        for item in itertools.product(*self._data_list):
            print(item)

def range_query_lamda_MEOM(lamda,query_lamda_dim,one_ans,lamda_ans,str1):
    if str1 =='ndr':
        member_cn = member1
    elif str1 =='dr':
        member_cn = member
    elif str1 =='lhio':
        member_cn = member_LHIO
    col = 2**lamda  # 计算矩阵的行数
    domain = []
    for i in range(lamda):
        domain.append(2)
    A1 = []
    for i in range(len(domain)):
        for j in range(domain[i]):
            a = np.zeros(domain, dtype=float)
            s = cartesian()
            for k in domain:
                s.add_data(range(0, k))
            for k in itertools.product(*s._data_list):
                if k[i] == j:
                    a[k] = 1
            A1.append(a.flatten())
    A1 = np.asarray(A1)
    A1 = matrix(A1)

    A2 = []
    for i in range(len(domain)):
        for m in range(i+1,len(domain)):
            for j in range(domain[i]):
                a = np.zeros(domain, dtype=float)
                s = cartesian()
                for k in domain:
                    s.add_data(range(0, k))
                for k in itertools.product(*s._data_list):
                    if k[i] == j and k[m] == j:
                        a[k] = 1
                A2.append(a.flatten())
    A2 = np.asarray(A2)
    A2 = matrix(A2)
    A3 = matrix(1, (1, col), 'd')
    b3 = matrix([1.0])

    ans = np.zeros((len(query_lamda_dim),epsilon_dim),dtype=float)
    for q in range(len(query_lamda_dim)):
        for e in range(epsilon_dim):
            b1 = []
            for n in range(2*lamda):
                if n%2 == 0:
                    b1.append((one_ans[q*lamda+n//2][e])/member_cn[e])
                else:
                    b1.append((member_cn[e]-one_ans[q*lamda+n//2][e])/member_cn[e])
            b2 = []
            for n in range(2*int(comb(lamda,2))):
                if n%2 == 0:
                    b2.append((lamda_ans[q*lamda+n//2][e])/member_cn[e])
                else:
                    b2.append((member_cn[e]-lamda_ans[q*lamda+n//2][e])/member_cn[e])
            b1 = matrix(b1)
            b2 = matrix(b2)

            A = matrix([A1, A3])
            b = matrix([b1, b3])
            def F(x=None, z=None):
                # print(x)
                if x is None: return 0, matrix(1.0, (col, 1))
                # implicit constraint that x should be non-negative
                if min(x) <= 0: return None
                f = x.T * log(x)
                grad = 1.0 + log(x)
                if z is None: return f, grad.T
                H = spdiag(z[0] * x ** -1)
                return f, grad.T, H

            sol = solvers.cp(F, G=A, h=b, A=A3, b=b3)
            p = sol['x']
            print(p)
            p = p*member_cn[e]
            ans[q][e] = p[0]
    return ans

In [ ]:
def range_query_lamda_LASSO(lamda,query_lamda_dim,one_ans,lamda_ans,str1):
    if str1 =='ndr':
        member_cn = member1
    elif str1 =='dr':
        member_cn = member
    domain = []
    for i in range(lamda):
        domain.append(2)
    matrix_tmp = []
    for i in range(len(domain)):
        for m in range(i+1,len(domain)):
            for j in range(domain[i]):
                a = np.zeros(domain, dtype=float)
                s = cartesian()
                for n in domain:
                    s.add_data(range(0, n))
                for n in itertools.product(*s._data_list):
                    if n[i] == j and n[m] == j:
                        a[n] = 1
                matrix_tmp.append(a.flatten())
    matrix_tmp = np.asarray(matrix_tmp)
    matrix = matrix_tmp[::2]
    matrix = np.transpose(matrix)
    matrix = np.linalg.pinv(matrix)
    ans = np.zeros((len(query_lamda_dim),epsilon_dim),dtype=float)
    for q in range(len(query_lamda_dim)):
        for e in range(epsilon_dim):
            b1 = []
            for n in range(lamda):
                b1.append(lamda_ans[q*lamda+n//2][e])
            rea = np.matmul(b1, matrix)
            ans[q][e] = rea[0]
    return ans

In [ ]:
def range_query_lamda(lamda,query_lamda_dim,lamda_ans,str1):
    if str1 =='ndr':
        member_cn = member1
    elif str1 =='dr':
        member_cn = member
    dim_list = []
    for i in range(lamda):
        dim_list.append(2)
    dim_tuple = tuple(dim_list)
    ans = np.zeros((len(query_lamda_dim),epsilon_dim),dtype=float)
    for q in range(len(query_lamda_dim)):
        for e in range(epsilon_dim):
            vector_z = np.full(dim_tuple,member1[e]/(2**lamda),dtype=float)
            interation_max = 100
            for i in range(interation_max):
                tmp_z = copy.deepcopy(vector_z)
                inx = 0
                for m in range(lamda):
                    for n in range(m+1,lamda):
                        tmp_slice = []
                        for l in range(lamda):
                            tmp_slice.append(':')
                        tmp_slice[m] = '0'
                        tmp_slice[n] = '0'
                        tmp_slice_update = ''
                        for l in range(lamda):
                            tmp_slice_update += tmp_slice[l]
                            if l < lamda - 1:
                                tmp_slice_update += ','
                        sentence1 = 'z_local = vector_z[' + tmp_slice_update + ']'
                        exec(sentence1)
                        slice_z = locals()['z_local']
                        tmp_sum = np.sum(slice_z)

                        if tmp_sum == 0:
                            pass
                        else:
                            slice_z = slice_z / tmp_sum * lamda_ans[q*lamda+inx][e]
                            sentence2 = 'vector_z[' + tmp_slice_update + '] = slice_z'
                            exec(sentence2)
                        inx += 1
                diff = np.sum(np.abs(tmp_z-vector_z))
                if diff < 1:
                    break
            tmp_slice_ans = []
            for i in range(lamda):
                tmp_slice_ans.append(0)
            tmp_slice_ans_tuple = tuple(tmp_slice_ans)
            # vector_z = vector_z/np.sum(vector_z)*member_cn[e]
            ans[q][e] = vector_z[tmp_slice_ans_tuple]
    return ans

In [ ]:
#build respond matrix,only used for partial grids
def build_matrix(one_grid,two_grid,i,j,str1):
    matrix_final = []
    if str1 =='ndr':
        member_ba = member1
    elif str1 =='dr':
        member_ba = member
    g1 = choose_g('HDG','g1',total_dim)
    g2 = choose_g('HDG','g2',total_dim)
    attributes = ['attribute' + str(inx) for inx in range(total_dim)]
    for e in range(epsilon_dim):
        tmp_one_gridi = one_grid[i][e]
        tmp_one_gridj = one_grid[j][e]
        G = index[i][j]
        tmp_two_grid = two_grid[G][e]
        matrix = np.full((c_domain,c_domain),member_ba[e]/(c_domain*c_domain))
        iteration_max = 100
        for iterta in range(iteration_max):
            matrix_tmp = copy.deepcopy(matrix)
            length = c_domain//g1[e]
            for cell in range(len(tmp_one_gridi)):  #Gi
                s = 0
                s = s + np.sum(matrix[cell*length:cell*length+length,:])
                if s != 0:
                    matrix[cell*length:cell*length+length,:] = matrix[cell*length:cell*length+length,:]/s*tmp_one_gridi[cell]
                matrix = matrix/np.sum(matrix)*member_ba[e]
            width = c_domain//g1[e]
            for cell in range(len(tmp_one_gridj)):  #Gj
                s = 0
                s = s + np.sum(matrix[:,cell*width:cell*width+width])
                if s != 0:
                    matrix[:,cell*width:cell*width+width] = matrix[:,cell*width:cell*width+width]/s*tmp_one_gridj[cell]
                matrix = matrix/np.sum(matrix)*member_ba[e]
            length = c_domain//g2[e]
            width = c_domain//g2[e]
            for cell1 in range(g2[e]):  #Gij
                for cell2 in range(g2[e]):
                    s = 0
                    s = s + np.sum(matrix[cell1*length:cell1*length+length,cell2*width:cell2*width+width])
                    if s != 0:
                        matrix[cell1*length:cell1*length+length,cell2*width:cell2*width+width] = matrix[cell1*length:cell1*length+length,cell2*width:cell2*width+width]/s*tmp_two_grid[cell1*g2[e]+cell2]
                    matrix = matrix/np.sum(matrix)*member_ba[e]
            diff = np.sum(np.abs(matrix-matrix_tmp))
            if diff < 1:
                break
        # matrix_trans = np.zeros((g2[e],g2[e]))
        # for inxi in range(g2[e]):
        #     for inxj in range(g2[e]):
        #         matrix_trans[inxi][inxj] = np.sum(matrix[inxi*length:inxi*length+length,inxj*width:inxj*width+width])
        matrix_final.append(matrix)
    return matrix_final

def range_query_two_HDG(dim,one_grid,two_grid,str1):

    attributes = ['attribute' + str(i) for i in range(total_dim)]
    one_range_res = []
    two_range_res = []
    if str1 == 'ndr':
        member_cn = member1
    else:
        member_cn = member
    for q in range(len(dim)):
        #ONE_ANS
        for i in range(len(dim.columns)//2):
            a = attributes.index(dim['attribute'+str(i)][q])
            range1 = str(dim['range'+str(i)][q]).split(',')
            range1 = list(map(int, range1))
            g1 = choose_g('HDG','g1',total_dim)
            res = np.zeros(epsilon_dim,dtype=float)
            for e in range(epsilon_dim):
                sum = 0
                length2 = g1[e]
                if len(range1) == 1:
                    res[e] = one_grid[a][e][int(range1[0])]
                else:
                    c = c_domain
                    block = choose_g('HDG','g1',total_dim)
                    block = np.round(c/block)
                    long1 = range1[1]-range1[0]+1
                    if c%block[e] != 0 and range1[1]//block[e] == g1[e]-1:
                        last = c%block[e]
                    else:
                        last = block[e]
                    if range1[1]//block[e] - range1[0]//block[e] == 0:
                        sum = sum + long1/block[e]*one_grid[a][e][int(range1[0]//block[e])]
                    else:
                        left  = range1[0]//block[e]
                        right = range1[1]//block[e]
                        num_grid = int(right - left + 1)
                        sum = sum + ((left+1)*block[e]-range1[0])/block[e]*one_grid[a][e][int(left)]
                        if num_grid > 2:
                            for inx in range(1,num_grid-1):
                                sum = sum + one_grid[a][e][int((left+inx))]
                        sum = sum + (range1[1]+1-right*block[e])/min(last,block[e])*one_grid[a][e][int(right)]
                    res[e] = sum
            one_range_res.append(res)
        #TWO_ANS
        for i in range(len(dim.columns)//2):
            for j in range(i+1,len(dim.columns)//2):
                a = attributes.index(dim['attribute'+str(i)][q])
                b = attributes.index(dim['attribute'+str(j)][q])
                G = index[a][b]
                Mij = build_matrix(one_grid,two_grid,a,b,str1)
                range1 = str(dim['range'+str(i)][q]).split(',')
                range2 = str(dim['range'+str(j)][q]).split(',')
                range1 = list(map(int, range1))
                range2 = list(map(int, range2))
                g2 = choose_g('HDG','g2',total_dim)
                #HDG
                res = np.zeros(epsilon_dim,dtype=float)
                for e in range(epsilon_dim):
                    sum = 0
                    length2 = g2[e]
                    if len(range1) == 1 and len(range2) == 1:#ord+ord
                        res[e] = Mij[e][range1[0]][range2[0]]
                    elif len(range1) == 2 and len(range2) == 1:#num+ord
                        for inx in range(range1[0],range1[1]+1):
                            sum = sum + Mij[e][inx][range2[0]]
                        res[e] = sum
                    elif len(range1) == 1 and len(range2) == 2:#ord+num
                        for inx in range(range2[0],range2[1]+1):
                            sum = sum + Mij[e][range1[0]][inx]
                        res[e] = sum
                    else:#num+num
                        for inx1 in range(range1[0],range1[1]+1):
                            for inx2 in range(range2[0],range2[1]+1):
                                sum = sum + Mij[e][inx1][inx2]
                        res[e] = sum
                two_range_res.append(res)
    return one_range_res,two_range_res

In [ ]:
for time_circle in range(10):
    true_one_grid_HDG,one_grid_n_HDG,one_group_n_HDG,true_two_grid_HDG,two_grid_n_HDG,two_group_n_HDG = construct_HDG(total_dim,transdf)
    one_group_HDG = copy.deepcopy(one_group_n_HDG)
    two_group_HDG = copy.deepcopy(two_group_n_HDG)
    true_two_grid_LHIO,two_grid_n_LHIO = construct_LHIO(total_dim,transdf)
    for time in range(100):
        one_grid_cn_HDG,two_grid_cn_HDG = consistency(one_grid_n_HDG,two_grid_n_HDG,'ndr')
        # one_grid_cn_CORR,two_grid_cn_CORR = consistency(one_grid_n_CORR,two_grid_n_CORR,'ndr')
        one_group_cn_HDG,two_group_cn_HDG = consistency(one_group_n_HDG,two_group_n_HDG,'dr')
        # one_group_cn_CORR,two_group_cn_CORR = consistency(one_group_n_CORR,two_group_CORR,'dr')
    lamda_HDG = []
    lamda_LHIO = []
    lamda_ours = []
    lamda_nonwgt = []
    lamda_dim = 2
    query_two_dim = range_query_list(lamda_dim) 
    print(query_two_dim)
    real_list_ans = real_ans(lamda_dim,query_two_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_two_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)
    # real_query_ans = [0.5088222314086521, 0.44478878038970326, 0.4736641192944822, 0.21918666577285187, 0.10583139673459659, 0.5191443957811941, 0.22193235311643428, 0.19185734715766894, 0.3605648909545942, 0.1407446408354189]
    print(real_query_ans)
    one_HDG_cn,three_HDG_cn = range_query_two_HDG(query_two_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,three_HDG_group = range_query_two_HDG(query_two_dim,one_group_cn_HDG,two_group_cn_HDG,'dr')

    three_LHIO_cn = range_query_two_LHIO(query_two_dim,two_grid_n_LHIO)
    a_HDG_MAE2 = []
    a_LHIO_MAE2 = []
    w_group_CORR_MEOM_MAE2 = []
    a_group_CORR_MEOM_MAE2 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_two_dim,two_grid_cn_HDG,three_HDG_cn,'average','ndr')/total_n
        LHIO_res = AC(inx,query_two_dim,two_grid_n_LHIO,three_LHIO_cn,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_two_dim,two_group_HDG,three_HDG_group,'weighted','dr')/total_n
        group_CORR_wgt_res = AC(inx,query_two_dim,two_group_HDG,three_HDG_group,'average','dr')/total_n


        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE2.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE2.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE2.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        a_group_CORR_MEOM_MAE2.append(mean_absolute_error(real_query_ans, group_CORR_wgt_res))
    lamda_HDG.append(a_HDG_MAE2[4])
    lamda_LHIO.append(a_LHIO_MAE2[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE2[4])
    lamda_nonwgt.append(a_group_CORR_MEOM_MAE2[4])

    lamda_dim = 3
    query_three_dim = range_query_list(lamda_dim) 
    print(query_three_dim)
    real_list_ans = real_ans(lamda_dim,query_three_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_three_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.09128037108211179, 0.2782780657847694, 0.17000776501608866, 0.11430541279346919, 0.0896342621558813, 0.11162443391729233, 0.08355771809081158, 0.07689181489095459, 0.12663772941246573, 0.07918966377666548]
    print(real_query_ans)

    one_HDG_cn,three_HDG_cn = range_query_two_HDG(query_three_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,three_HDG_group = range_query_two_HDG(query_three_dim,one_group_HDG,two_group_HDG,'dr')

    three_LHIO_cn = range_query_two_LHIO(query_three_dim,two_grid_n_LHIO)

    #HDG
    three_HDG_cn_ans = range_query_lamda(lamda_dim,query_three_dim,three_HDG_cn,'ndr')
    #LHIO
    three_LHIO_cn_ans = range_query_lamda(lamda_dim,query_three_dim,three_LHIO_cn,'ndr')
    #OURS
    three_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_three_dim,one_HDG_group,three_HDG_group,'dr')
    # #non-group
    # three_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_three_dim,one_HDG_cn,three_HDG_cn,'ndr')
    # #non-MEOM
    # three_non_MEOM_ans = range_query_lamda(lamda_dim,query_three_dim,three_HDG_group,'dr')


    a_HDG_MAE3 = []
    a_LHIO_MAE3 = []
    w_group_CORR_MEOM_MAE3 = []
    # w_non_group_MAE3 = []
    # w_non_MEOM_MAE3 = []
    a_non_wgt_MAE3 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_three_dim,two_grid_cn_HDG,three_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_three_dim,two_grid_n_LHIO,three_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_three_dim,two_group_HDG,three_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_three_dim,two_grid_cn_HDG,three_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_three_dim,two_group_HDG,three_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_three_dim,two_group_HDG,three_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE3.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE3.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE3.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE3.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE3.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE3.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE3[4])
    lamda_LHIO.append(a_LHIO_MAE3[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE3[4])
    lamda_nonwgt.append(a_non_wgt_MAE3[4])

    lamda_dim = 4
    query_four_dim = range_query_list(lamda_dim) 
    print(query_four_dim)
    real_list_ans = real_ans(lamda_dim,query_four_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_four_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.0699866300202598, 0.07687645244905256, 0.047293510159694914, 0.12263092524728876, 0.09164627651650578, 0.09201497512215469, 0.11681321505184127, 0.06976317631986652, 0.0579322339411274, 0.05451851313907758]
    print(real_query_ans)

    one_HDG_cn,four_HDG_cn = range_query_two_HDG(query_four_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,four_HDG_group = range_query_two_HDG(query_four_dim,one_group_HDG,two_group_HDG,'dr')

    four_LHIO_cn = range_query_two_LHIO(query_four_dim,two_grid_n_LHIO)

    #HDG
    four_HDG_cn_ans = range_query_lamda(lamda_dim,query_four_dim,four_HDG_cn,'ndr')
    #LHIO
    four_LHIO_cn_ans = range_query_lamda(lamda_dim,query_four_dim,four_LHIO_cn,'ndr')
    #OURS
    four_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_four_dim,one_HDG_group,four_HDG_group,'dr')
    # #non-group
    # four_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_four_dim,one_HDG_cn,four_HDG_cn,'ndr')
    # #non-MEOM
    # four_non_MEOM_ans = range_query_lamda(lamda_dim,query_four_dim,four_HDG_group,'dr')


    a_HDG_MAE4 = []
    a_LHIO_MAE4 = []
    w_group_CORR_MEOM_MAE4 = []
    # w_non_group_MAE = []
    # w_non_MEOM_MAE = []
    a_non_wgt_MAE4 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_four_dim,two_grid_cn_HDG,four_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_four_dim,two_grid_n_LHIO,four_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_four_dim,two_group_HDG,four_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_four_dim,two_grid_cn_HDG,four_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_four_dim,two_group_HDG,four_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_four_dim,two_group_HDG,four_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE4.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE4.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE4.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE4.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE4.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE4.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE4[4])
    lamda_LHIO.append(a_LHIO_MAE4[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE4[4])
    lamda_nonwgt.append(a_non_wgt_MAE4[4])

    lamda_dim = 5
    query_five_dim = range_query_list(lamda_dim) 
    print(query_five_dim)
    real_list_ans = real_ans(lamda_dim,query_five_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_five_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.023657695000595877, 0.03675673712906686, 0.03526890790728161, 0.055193995054224766, 0.03769989795614349, 0.043095839292098675, 0.03098232108807055, 0.017710102341794782, 0.02885811434870695, 0.02324197801215588]
    print(real_query_ans)

    one_HDG_cn,five_HDG_cn = range_query_two_HDG(query_five_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,five_HDG_group = range_query_two_HDG(query_five_dim,one_group_HDG,two_group_HDG,'dr')

    five_LHIO_cn = range_query_two_LHIO(query_five_dim,two_grid_n_LHIO)

    #HDG
    five_HDG_cn_ans = range_query_lamda(lamda_dim,query_five_dim,five_HDG_cn,'ndr')
    #LHIO
    five_LHIO_cn_ans = range_query_lamda(lamda_dim,query_five_dim,five_LHIO_cn,'ndr')
    #OURS
    five_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_five_dim,one_HDG_group,five_HDG_group,'dr')
    # #non-group
    # five_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_five_dim,one_HDG_cn,five_HDG_cn,'ndr')
    # #non-MEOM
    # five_non_MEOM_ans = range_query_lamda(lamda_dim,query_five_dim,five_HDG_group,'dr')


    a_HDG_MAE5 = []
    a_LHIO_MAE5 = []
    w_group_CORR_MEOM_MAE5 = []
    # w_non_group_MAE5 = []
    # w_non_MEOM_MAE5 = []
    a_non_wgt_MAE5 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_five_dim,two_grid_cn_HDG,five_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_five_dim,two_grid_n_LHIO,five_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_five_dim,two_group_HDG,five_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_five_dim,two_grid_cn_HDG,five_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_five_dim,two_group_HDG,five_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_five_dim,two_group_HDG,five_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE5.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE5.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE5.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE5.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE5.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE5.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE5[4])
    lamda_LHIO.append(a_LHIO_MAE5[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE5[4])
    lamda_nonwgt.append(a_non_wgt_MAE5[4])

    lamda_dim = 6
    query_six_dim = range_query_list(lamda_dim) 
    print(query_six_dim)
    real_list_ans = real_ans(lamda_dim,query_six_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_six_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.010790486086282922, 0.007452180908115839, 0.012631185943272553, 0.01423400071505184, 0.010077296359194375, 0.008889267518770111, 0.008254752115361697, 0.008427928733166487, 0.032026501608866643, 0.028576004051960435]
    print(real_query_ans)

    one_HDG_cn,six_HDG_cn = range_query_two_HDG(query_six_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,six_HDG_group = range_query_two_HDG(query_six_dim,one_group_HDG,two_group_HDG,'dr')

    six_LHIO_cn = range_query_two_LHIO(query_six_dim,two_grid_n_LHIO)

    #HDG
    six_HDG_cn_ans = range_query_lamda(lamda_dim,query_six_dim,six_HDG_cn,'ndr')
    #LHIO
    six_LHIO_cn_ans = range_query_lamda(lamda_dim,query_six_dim,six_LHIO_cn,'ndr')
    #OURS
    six_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_six_dim,one_HDG_group,six_HDG_group,'dr')
    # #non-group
    # six_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_six_dim,one_HDG_cn,six_HDG_cn,'ndr')
    # #non-MEOM
    # six_non_MEOM_ans = range_query_lamda(lamda_dim,query_six_dim,six_HDG_group,'dr')

    a_HDG_MAE6 = []
    a_LHIO_MAE6 = []
    w_group_CORR_MEOM_MAE6 = []
    # w_non_group_MAE6 = []
    # w_non_MEOM_MAE6 = []
    a_non_wgt_MAE6 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_six_dim,two_grid_n_LHIO,six_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE6.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE6.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE6.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE6.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE6.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE6.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE6[4])
    lamda_LHIO.append(a_LHIO_MAE6[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE6[4])
    lamda_nonwgt.append(a_non_wgt_MAE6[4])

    lamda_dim = 7
    query_seven_dim = range_query_list(lamda_dim) 
    print(query_seven_dim)
    real_list_ans = real_ans(lamda_dim,query_seven_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_seven_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.0029281745322369203, 0.005063088428077702, 0.0022736414015016087, 0.000448769514956501, 0.005980645185317602, 0.005505806071981885, 0.006211547342390657, 0.000909642772017638, 0.011264859671076153, 0.01725434989870099]
    print(real_query_ans)

    one_HDG_cn,seven_HDG_cn = range_query_two_HDG(query_seven_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,seven_HDG_group = range_query_two_HDG(query_seven_dim,one_group_HDG,two_group_HDG,'dr')

    seven_LHIO_cn = range_query_two_LHIO(query_seven_dim,two_grid_n_LHIO)

    #HDG
    seven_HDG_cn_ans = range_query_lamda(lamda_dim,query_seven_dim,seven_HDG_cn,'ndr')
    #LHIO
    seven_LHIO_cn_ans = range_query_lamda(lamda_dim,query_seven_dim,seven_LHIO_cn,'ndr')
    #OURS
    seven_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_seven_dim,one_HDG_group,seven_HDG_group,'dr')
    # #non-group
    # seven_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_seven_dim,one_HDG_cn,seven_HDG_cn,'ndr')
    # #non-MEOM
    # seven_non_MEOM_ans = range_query_lamda(lamda_dim,query_seven_dim,seven_HDG_group,'dr')


    a_HDG_MAE7 = []
    a_LHIO_MAE7 = []
    w_group_CORR_MEOM_MAE7 = []
    # w_non_group_MAE7 = []
    # w_non_MEOM_MAE7 = []
    a_non_wgt_MAE7 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_six_dim,two_grid_n_LHIO,six_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE7.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE7.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE7.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE7.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE7.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE7.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE7[4])
    lamda_LHIO.append(a_LHIO_MAE7[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE7[4])
    lamda_nonwgt.append(a_non_wgt_MAE7[4])

    lamda_dim = 8
    query_eight_dim = range_query_list(lamda_dim) 
    print(query_eight_dim)
    real_list_ans = real_ans(lamda_dim,query_eight_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_eight_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.000454821386008819, 0.002584148939339769, 0.00020576361577881062, 0.0010232317363842213, 8.705383744488142e-05, 0.0009822652246454535, 0.003474239512572995, 0.0014026374985103088, 0.002677254647836968, 0.00023369532832797043]
    print(real_query_ans)

    one_HDG_cn,eight_HDG_cn = range_query_two_HDG(query_eight_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,eight_HDG_group = range_query_two_HDG(query_eight_dim,one_group_HDG,two_group_HDG,'dr')

    eight_LHIO_cn = range_query_two_LHIO(query_eight_dim,two_grid_n_LHIO)

    #HDG
    eight_HDG_cn_ans = range_query_lamda(lamda_dim,query_eight_dim,eight_HDG_cn,'ndr')
    #LHIO
    eight_LHIO_cn_ans = range_query_lamda(lamda_dim,query_eight_dim,eight_LHIO_cn,'ndr')
    #OURS
    eight_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_eight_dim,one_HDG_group,eight_HDG_group,'dr')
    # #non-group
    # eight_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_eight_dim,one_HDG_cn,eight_HDG_cn,'ndr')
    # #non-MEOM
    # eight_non_MEOM_ans = range_query_lamda(lamda_dim,query_eight_dim,eight_HDG_group,'dr')

    a_HDG_MAE8 = []
    a_LHIO_MAE8 = []
    w_group_CORR_MEOM_MAE8 = []
    # w_non_group_MAE8 = []
    # w_non_MEOM_MAE8 = []
    a_non_wgt_MAE8 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_six_dim,two_grid_n_LHIO,six_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE8.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE8.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE8.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE8.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE8.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE8.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE8[4])
    lamda_LHIO.append(a_LHIO_MAE8[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE8[4])
    lamda_nonwgt.append(a_non_wgt_MAE8[4])

    lamda_dim = 9
    query_nine_dim = range_query_list(lamda_dim) 
    print(query_nine_dim)
    real_list_ans = real_ans(lamda_dim,query_nine_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_nine_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.000150365719222977, 0.00011545107853652723, 4.003545465379573e-05, 0.0005590997795256823, 0.00013174457752353712, 2.3276427124299846e-05, 4.841496841854368e-05, 0.0001806250744845668, 0.0001331411631509951, 3.956992611130974e-05]
    print(real_query_ans)

    one_HDG_cn,nine_HDG_cn = range_query_two_HDG(query_nine_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,nine_HDG_group = range_query_two_HDG(query_nine_dim,one_group_HDG,two_group_HDG,'dr')

    nine_LHIO_cn = range_query_two_LHIO(query_nine_dim,two_grid_n_LHIO)

    #HDG
    nine_HDG_cn_ans = range_query_lamda(lamda_dim,query_nine_dim,nine_HDG_cn,'ndr')
    #LHIO
    nine_LHIO_cn_ans = range_query_lamda(lamda_dim,query_nine_dim,nine_LHIO_cn,'ndr')
    #OURS
    nine_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_nine_dim,one_HDG_group,nine_HDG_group,'dr')
    # #non-group
    # nine_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_nine_dim,one_HDG_cn,nine_HDG_cn,'ndr')
    # #non-MEOM
    # nine_non_MEOM_ans = range_query_lamda(lamda_dim,query_nine_dim,nine_HDG_group,'dr')

    a_HDG_MAE9 = []
    a_LHIO_MAE9 = []
    w_group_CORR_MEOM_MAE9 = []
    # w_non_group_MAE9 = []
    # w_non_MEOM_MAE9 = []
    a_non_wgt_MAE9 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_six_dim,two_grid_n_LHIO,six_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE9.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE9.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE9.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE9.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE9.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE9.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE9[4])
    lamda_LHIO.append(a_LHIO_MAE9[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE9[4])
    lamda_nonwgt.append(a_non_wgt_MAE9[4])

    lamda_dim = 10
    query_ten_dim = range_query_list(lamda_dim) 
    print(query_ten_dim)
    real_list_ans = real_ans(lamda_dim,query_ten_dim,transdf,transdf[epsilon_type])
    real_query_ans = []
    for q in range(len(query_ten_dim)):
        real_query_ans.append(sum(real_list_ans[q])/total_n)

    # real_query_ans = [0.000149900190680491, 0.000151762304850435, 0.00036683649147896554, 0.0004259586163746872, 0.00032400786557025383, 0.0002565062269097843, 3.1190412346561794e-05, 4.329415445119771e-05, 0.0001778319032296508, 0.00018853905970682875]
    print(real_query_ans)

    one_HDG_cn,ten_HDG_cn = range_query_two_HDG(query_ten_dim,one_grid_cn_HDG,two_grid_cn_HDG,'ndr')
    one_HDG_group,ten_HDG_group = range_query_two_HDG(query_ten_dim,one_group_HDG,two_group_HDG,'dr')

    ten_LHIO_cn = range_query_two_LHIO(query_ten_dim,two_grid_n_LHIO)

    #HDG
    ten_HDG_cn_ans = range_query_lamda(lamda_dim,query_ten_dim,ten_HDG_cn,'ndr')
    #LHIO
    ten_LHIO_cn_ans = range_query_lamda(lamda_dim,query_ten_dim,ten_LHIO_cn,'ndr')
    #OURS
    ten_MEOM_group_ans = range_query_lamda_MEOM(lamda_dim,query_ten_dim,one_HDG_group,ten_HDG_group,'dr')
    # #non-group
    # ten_non_group_ans = range_query_lamda_MEOM(lamda_dim,query_ten_dim,one_HDG_cn,ten_HDG_cn,'ndr')
    # #non-MEOM
    # ten_non_MEOM_ans = range_query_lamda(lamda_dim,query_ten_dim,ten_HDG_group,'dr')

    a_HDG_MAE10 = []
    a_LHIO_MAE10 = []
    w_group_CORR_MEOM_MAE10 = []
    # w_non_group_MAE10 = []
    # w_non_MEOM_MAE10 = []
    a_non_wgt_MAE10 = []


    #Weighted
    for inx in range(1,11):
        HDG_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_HDG_cn_ans,'average','ndr')/total_n
        LHIO_res = AC(inx,query_six_dim,two_grid_n_LHIO,six_LHIO_cn_ans,'average','lhio')/total_n
        group_CORR_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'weighted','dr')/total_n
        # non_group_res = AC(inx,query_six_dim,two_grid_cn_HDG,six_non_group_ans,'weighted','ndr')/total_n
        # non_MEOM_res = AC(inx,query_six_dim,two_group_HDG,six_non_MEOM_ans,'weighted','dr')/total_n
        non_wgt_res = AC(inx,query_six_dim,two_group_HDG,six_MEOM_group_ans,'average','dr')/total_n

        # print(n_TDG_res,cn_TDG_res,n_HDG_res,cn_HDG_res)
        
        a_HDG_MAE10.append(mean_absolute_error(real_query_ans, HDG_res))
        a_LHIO_MAE10.append(mean_absolute_error(real_query_ans, LHIO_res))
        w_group_CORR_MEOM_MAE10.append(mean_absolute_error(real_query_ans, group_CORR_MEOM_res))
        # w_non_group_MAE10.append(mean_absolute_error(real_query_ans, non_group_res))
        # w_non_MEOM_MAE10.append(mean_absolute_error(real_query_ans, non_MEOM_res))
        a_non_wgt_MAE10.append(mean_absolute_error(real_query_ans, non_wgt_res))
    lamda_HDG.append(a_HDG_MAE10[4])
    lamda_LHIO.append(a_LHIO_MAE10[4])
    lamda_ours.append(w_group_CORR_MEOM_MAE10[4])
    lamda_nonwgt.append(a_non_wgt_MAE10[4])

    mae_values = {
        "lamda_HDG" : lamda_HDG,
        "lamda_LHIO" : lamda_LHIO,
        "lamda_ours" : lamda_ours,
        "lamda_nonwgt" : lamda_nonwgt,
        # 添加更多的MAE值，格式为 "MAE名称": [MAE值列表]
    }

    for mae_name, values in mae_values.items():
        filename = mae_name + ".txt"
        with open(filename, "a+") as file:
            row = values
            row_str = " ".join(str(value) for value in row)
            file.write(row_str + "\n")
    print(time_circle)